In [1]:
import mph
import re
import numpy as np
from scipy import integrate
import pandas as pd
import matplotlib.pyplot as plt
from structure import Solve, db
from tqdm import tqdm as _tqdm
from itertools import product
import peewee

from solver import (
    combinations_dict,
    evaluate_species,
    sweep,
    get_solves,
    solve_to_sql,
    solves_to_sql,
    model_parametrs
)


In [ ]:
a = {
    'Ke': 1000000000.0,
    'KH': 1000000000.0,
    'Kr': 1000000000.0,
    'Kdisp': 1000000000.0,
    'KqH': 2000.0,
    'Ks': 2.0,
    'Kd': 0.05,
    'Kc': 1.0,
    'Kp': 0.001,
    'KrD': 1000000000.0,
    'Kph': 1e-05,
    'light': 1e-05,
}
a = list(a.keys())
b = [
    f"{str(i)} = DecimalField(max_digits=20, decimal_places=5, default =0)"
    for i in a
]

string = '\n'.join(b)
print(string)

Ke = DecimalField(max_digits=20, decimal_places=5, default =0)
KH = DecimalField(max_digits=20, decimal_places=5, default =0)
Kr = DecimalField(max_digits=20, decimal_places=5, default =0)
Kdisp = DecimalField(max_digits=20, decimal_places=5, default =0)
KqH = DecimalField(max_digits=20, decimal_places=5, default =0)
Ks = DecimalField(max_digits=20, decimal_places=5, default =0)
Kd = DecimalField(max_digits=20, decimal_places=5, default =0)
Kc = DecimalField(max_digits=20, decimal_places=5, default =0)
Kp = DecimalField(max_digits=20, decimal_places=5, default =0)
KrD = DecimalField(max_digits=20, decimal_places=5, default =0)
Kph = DecimalField(max_digits=20, decimal_places=5, default =0)
light = DecimalField(max_digits=20, decimal_places=5, default =0)


In [2]:
client = mph.start()
model = client.load('System_batch.mph')

In [3]:
model.solve()

In [4]:
model.outer('Data')

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),
 array([0.e+00, 1.e-05, 2.e-05, 3.e-05, 4.e-05, 5.e-05, 6.e-05, 7.e-05,
        8.e-05, 9.e-05, 1.e-04]))

In [5]:
df = evaluate_species(model, [2,3])
df

,Time,Q,DH,Qm,DHp,QH,D,QHD,QHH,prod,QD,light
0,0.000000e+00,1.000000,1.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00001
1,7.812500e-10,0.999992,0.999992,0.000008,0.000008,4.768223e-11,4.768223e-11,0.000000e+00,0.000000e+00,7.812439e-15,3.725174e-20,0.00001
2,1.562500e-09,0.999984,0.999984,0.000016,0.000016,2.384023e-10,2.384023e-10,4.440287e-20,4.440280e-20,1.562486e-14,2.235006e-19,0.00001
3,3.125000e-09,0.999969,0.999969,0.000031,0.000031,1.763978e-09,1.763978e-09,4.906306e-18,4.906290e-18,3.125423e-14,2.979630e-18,0.00001
4,6.250000e-09,0.999938,0.999938,0.000062,0.000062,1.396350e-08,1.396350e-08,6.142171e-16,6.142132e-16,6.311159e-14,4.661286e-17,0.00001
...,...,...,...,...,...,...,...,...,...,...,...,...
358,1.932690e-04,0.402270,0.114060,0.000960,0.000960,7.122499e-04,5.249525e-04,3.077587e-01,2.882545e-01,2.883263e-01,4.382112e-05,0.00002
359,1.954724e-04,0.400869,0.112059,0.000950,0.000950,7.075737e-04,5.189048e-04,3.085747e-01,2.888545e-01,2.889266e-01,4.436693e-05,0.00002
360,1.976759e-04,0.399495,0.110099,0.000940,0.000940,7.029892e-04,5.129521e-04,3.093760e-01,2.894407e-01,2.895133e-01,4.491228e-05,0.00002
361,1.998794e-04,0.398149,0.108181,0.000931,0.000931,6.984939e-04,5.070920e-04,3.101631e-01,2.900136e-01,2.900866e-01,4.545717e-05,0.00002


In [ ]:
params = {
    key: float(value)
    for key,
    value in model.parameters().items()
    if 'light' not in key
}
params['light'] = 1.e-05


In [ ]:
solves_to_sql(model=model, name='test_sweep')

In [ ]:
solve_to_sql(df, params=params, name='test', desc='test desc')

In [7]:
variative_params = combinations_dict({
    'Ks':np.linspace(0.1,5.1,11).round(5),
    'Kc':np.linspace(0.1,2.1,11).round(5),
})

In [17]:
model.outer('Data')[1]

array([0.e+00, 1.e-05, 2.e-05, 3.e-05, 4.e-05, 5.e-05, 6.e-05, 7.e-05,
       8.e-05, 9.e-05, 1.e-04])

In [25]:
evaluate_species(model,outer_numbers=[1])

,Time,Q,DH,Qm,DHp,QH,D,QHD,QHH,prod,QD,light
0,0.000000e+00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0
1,2.000000e-07,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.000000e-12,0.0,0.0
2,4.000000e-07,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000e-12,0.0,0.0
3,8.000000e-07,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,8.000000e-12,0.0,0.0
4,1.600000e-06,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.600000e-11,0.0,0.0
5,3.200000e-06,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.200000e-11,0.0,0.0
6,6.400000e-06,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.400000e-11,0.0,0.0
7,1.280000e-05,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.280000e-10,0.0,0.0
8,2.560000e-05,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.560000e-10,0.0,0.0
9,4.560000e-05,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.560000e-10,0.0,0.0


In [8]:
sweep(
    model=model,
    name='sweep',
    desc='',
    variable_params=variative_params[:5],
)


Saving... : 100%|██████████| 5/5 [01:10<00:00, 14.02s/it]
